### Testing

In [5]:
import tensorflow as tf
LABEL_CLASS_LIST = ['NG', 'OK']
label_lookup = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(LABEL_CLASS_LIST, tf.constant([0, 1], dtype=tf.int64)), -1)
DEGREE_CLASS_LIST = ['000', '180', '270', '090']
# degree_lookup = tf.lookup.StaticHashTable(
#         tf.lookup.KeyValueTensorInitializer(DEGREE_CLASS_LIST, tf.constant(list(range(len(DEGREE_CLASS_LIST))), dtype=tf.int64)), -1)
degree_lookup = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(DEGREE_CLASS_LIST, tf.constant([0, 4, 6, 2], dtype=tf.int64)), -1)
mirrored_strategy = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
with mirrored_strategy.scope():
    METRICS = [
                tf.keras.metrics.CategoricalAccuracy(name='acc'),
                tf.keras.metrics.Precision(name="precision/ok", class_id=0),
                tf.keras.metrics.Precision(name="precision/ng", class_id=1),
                tf.keras.metrics.Recall(name="recall/ok", class_id=0),
                tf.keras.metrics.Recall(name="recall/ng", class_id=1),
            ]
BATCH_SIZE = 512
target_shape = (32, 32, 3)
LABEL_NUM = 2
# DEGREE_NUM = len(DEGREE_CLASS_LIST)
DEGREE_NUM = 8

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [4]:
def parse_img(img):
    img = tf.io.decode_image(img, channels=target_shape[-1], 
                                dtype=tf.dtypes.float32, expand_animations = False)
    img = tf.image.resize_with_pad(img, target_shape[1], target_shape[0])
    return img
# def parse_img(img):
#     img = tf.io.decode_jpeg(img, channels=target_shape[-1])
#     img = tf.image.resize_with_pad(img, target_shape[1], target_shape[0], tf.image.ResizeMethod.BILINEAR)
#     return img
def label_to_onehot(label):
    label = label_lookup.lookup(label)
    onehot_label = tf.one_hot(label, LABEL_NUM)
    onehot_label = tf.cast(onehot_label, dtype=tf.float32)
    return onehot_label
def degree_to_onehot(degree):
    degree = degree_lookup.lookup(degree)
    onehot_degree = tf.one_hot(degree, DEGREE_NUM)
#     onehot_degree = tf.cast(onehot_degree, dtype=tf.float32)
    return onehot_degree
def test_parse_func(path, degree, label):
    features = {
        'image': parse_img(tf.io.read_file(path)),
        'degree': degree_to_onehot(degree),
        'label': label_to_onehot(label),
    }
    return features, features['label']
def test_ds_to_eva(test_ds, batch_size=BATCH_SIZE):
    return test_ds.map(test_parse_func, tf.data.experimental.AUTOTUNE).batch(batch_size)

### TEST ALL METADATA

In [5]:
test_ds = tf.data.experimental.SqlDataset(
    "sqlite", "/tf/robertnb/p1-dip-metadata.db", 
    f"""select path, degree, label from metadata 
        where component_class = 'label' or
              component_class = 'screw' or
              component_class = 'heat_sink' 
    """, (tf.string, tf.string, tf.string))
print([i for i,_ in enumerate(test_ds)][-1] + 1)
test_ds = test_ds_to_eva(test_ds)

24564


#### ( first_tuned ) 

In [6]:
import glob
ft_pattern = '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/first_tuned/*/final'
ft = glob.glob(ft_pattern)

for p in ft:
    loaded = tf.keras.models.load_model(p)
    evaluate = loaded.evaluate(test_ds)
    named_results = dict(zip(loaded.metrics_names, evaluate))
    print(p, '\n', named_results)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


48/48 [==============================] - 17s 364ms/step - loss: 0.0797 - acc: 0.9829 - precision/ok: 0.8238 - precision/ng: 0.9907 - recall/ok: 0.8132 - recall/ng: 0.9913
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/first_tuned/op-Adadelta_lr-7_loop-2/final 
 {'loss': 0.07969395816326141, 'acc': 0.9828611016273499, 'precision/ok': 0.8237847089767456, 'precision/ng': 0.9906885623931885, 'recall/ok': 0.8131962418556213, 'recall/ng': 0.9913236498832703}
48/48 [==============================] - 22s 462ms/step - loss: 0.0797 - acc: 0.9829 - precision/ok: 0.8238 - precision/ng: 0.9907 - recall/ok: 0.8132 - recall/ng: 0.9913
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/first_tuned/op-SGD_lr-7_loop-13/final 
 {'loss': 0.07970903813838959, 'acc': 0.9828611016273499, 'precision/ok': 0.8237847089767456, 'precision/ng': 0.9906885623931885, 'recall/ok': 0.8131962418556213, 'recall/ng': 0.9913236498832703}
48/48 [==============================] - 19s 388ms/step - 

48/48 [==============================] - 17s 364ms/step - loss: 0.0801 - acc: 0.9827 - precision/ok: 0.8211 - precision/ng: 0.9907 - recall/ok: 0.8141 - recall/ng: 0.9912
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/first_tuned/op-adam_lr-8_loop-32/final 
 {'loss': 0.08011358976364136, 'acc': 0.9827389717102051, 'precision/ok': 0.8210890293121338, 'precision/ng': 0.9907292723655701, 'recall/ok': 0.8140531182289124, 'recall/ng': 0.9911527037620544}
48/48 [==============================] - 18s 368ms/step - loss: 0.0823 - acc: 0.9823 - precision/ok: 0.8119 - precision/ng: 0.9909 - recall/ok: 0.8175 - recall/ng: 0.9906
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/first_tuned/op-adam_lr-7_loop-11/final 
 {'loss': 0.0823264941573143, 'acc': 0.9823318719863892, 'precision/ok': 0.8119149208068848, 'precision/ng': 0.9908931255340576, 'recall/ok': 0.817480742931366, 'recall/ng': 0.9905543327331543}
48/48 [==============================] - 17s 358ms/step - loss

#### ( relabeled_vgg_with_tfrecords_r50 ) 

In [7]:
import glob
rvw_pattern = '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/relabeled_vgg_with_tfrecords_r50/*/final'
rvw = glob.glob(rvw_pattern)

for p in rvw:
    loaded = tf.keras.models.load_model(p)
    evaluate = loaded.evaluate(test_ds)
    named_results = dict(zip(loaded.metrics_names, evaluate))
    print(p, '\n', named_results)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


48/48 [==============================] - 22s 452ms/step - loss: 0.0969 - acc: 0.9758 - precision/ok: 0.7028 - precision/ng: 0.9925 - recall/ok: 0.8509 - recall/ng: 0.9820
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/relabeled_vgg_with_tfrecords_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.09615402259735728/final 
 {'loss': 0.09686160832643509, 'acc': 0.9758182764053345, 'precision/ok': 0.7027601003646851, 'precision/ng': 0.9924841523170471, 'recall/ok': 0.8508997559547424, 'recall/ng': 0.9820489883422852}
48/48 [==============================] - 21s 447ms/step - loss: 0.1213 - acc: 0.9649 - precision/ok: 0.6004 - precision/ng: 0.9891 - recall/ok: 0.7841 - recall/ng: 0.9740
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/relabeled_vgg_with_tfrecords_r50/b-VGG16_lr-5_op-adam_adv-True_ss-0.0877538779325646/final 
 {'loss': 0.12131451070308685, 'acc': 0.9649487137794495, 'precision/ok': 0.6003937125205994, 'precision/ng': 0.989062488079071, 'recall/ok': 0.7840616

48/48 [==============================] - 18s 379ms/step - loss: 0.0765 - acc: 0.9778 - precision/ok: 0.7349 - precision/ng: 0.9917 - recall/ok: 0.8338 - recall/ng: 0.9850
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/relabeled_vgg_with_tfrecords_r50/b-VGG16_lr-5_op-adam_adv-True_ss-0.0544650012577994/final 
 {'loss': 0.07646826654672623, 'acc': 0.9778130650520325, 'precision/ok': 0.7348942756652832, 'precision/ng': 0.9916523098945618, 'recall/ok': 0.8337618112564087, 'recall/ng': 0.9849980473518372}


#### ( dc0503_relabeled_trans_r50 )

In [9]:
def parse_example(example_proto):    
    image_feature_description = {
        "path": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "degree": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.string),
    }
    features_in_example = tf.io.parse_single_example(example_proto, image_feature_description)
    features = {
        'image': parse_img(tf.io.read_file(features_in_example['path'])),
        'degree': degree_to_onehot(features_in_example['degree']),
    }
    label = label_to_onehot(features_in_example['label'])
    return features, label

In [10]:
rotated_label = '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/tfrecord/rotated_ok_label_relabeled_ori_included.tfrecord'
rl_ds = tf.data.TFRecordDataset(rotated_label).map(parse_example, tf.data.experimental.AUTOTUNE).batch(all_var_dict['BATCH_SIZE'])


In [8]:
dc0503_h5_paths = [ 
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.054703700049706504/final',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.054703700049706504/ep_032-vl_0.003479-va_0.999625.h5',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.03082552306191831/final',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.03082552306191831/ep_021-vl_0.050832-va_0.990000.h5',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-RMSprop_adv-True_ss-0.017734547736807525/final',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-RMSprop_adv-True_ss-0.017734547736807525/ep_044-vl_0.004045-va_0.999750.h5',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-3_op-adagrad_adv-True_ss-0.10239825411296267/final',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-3_op-adagrad_adv-True_ss-0.10239825411296267/ep_091-vl_0.653964-va_0.836375.h5',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-DenseNet121_lr-3_op-adam_adv-True_ss-0.06601277020265413/final',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-DenseNet121_lr-3_op-adam_adv-True_ss-0.06601277020265413/ep_008-vl_0.677017-va_0.865000.h5',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-adagrad_adv-True_ss-0.16983435348865475/final',
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-adagrad_adv-True_ss-0.16983435348865475/ep_039-vl_0.009054-va_1.000000.h5',
]
for e in map(tf.keras.models.load_model, dc0503_h5_paths):
    e.evaluate(test_ds)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


17/17 [==============================] - 7s 397ms/step - loss: 0.1783 - acc: 0.9799 - precision/ok: 0.8889 - precision/ng: 0.9810 - recall/ok: 0.3548 - recall/ng: 0.9987


In [7]:
import glob
dc0503_pattern = '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/*/final'
dc0503 = glob.glob(dc0503_pattern)

for p in dc0503:
    loaded = tf.keras.models.load_model(p)
    evaluate = loaded.evaluate(test_ds)
    named_results = dict(zip(loaded.metrics_names, evaluate))
    print(p, '\n', named_results)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


17/17 [==============================] - 8s 483ms/step - loss: 0.7490 - acc: 0.0700 - precision/ok: 0.0303 - precision/ng: 0.9971 - recall/ok: 0.9960 - recall/ng: 0.0422
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-DenseNet121_lr-5_op-adagrad_adv-True_ss-0.10384481755287436/final 
 {'loss': 0.748958945274353, 'acc': 0.07001647353172302, 'precision/ok': 0.03031046688556671, 'precision/ng': 0.9971346855163574, 'recall/ok': 0.9959677457809448, 'recall/ng': 0.042181819677352905}
17/17 [==============================] - 7s 437ms/step - loss: 1.0232 - acc: 0.5450 - precision/ok: 0.0570 - precision/ng: 0.9966 - recall/ok: 0.9395 - recall/ng: 0.5331
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-DenseNet121_lr-3_op-RMSprop_adv-True_ss-0.18732260626976266/final 
 {'loss': 1.0231965780258179, 'acc': 0.5449517369270325, 'precision/ok': 0.05703794211149216, 'precision/ng': 0.9966009259223938, 'recall/ok': 0

17/17 [==============================] - 6s 360ms/step - loss: 0.1875 - acc: 0.9779 - precision/ok: 0.7308 - precision/ng: 0.9817 - recall/ok: 0.3831 - recall/ng: 0.9958
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.054703700049706504/final 
 {'loss': 0.18751294910907745, 'acc': 0.977877140045166, 'precision/ok': 0.7307692170143127, 'precision/ng': 0.9817160367965698, 'recall/ok': 0.38306450843811035, 'recall/ng': 0.9957575798034668}
17/17 [==============================] - 8s 444ms/step - loss: 1.2271 - acc: 0.6339 - precision/ok: 0.0425 - precision/ng: 0.9786 - recall/ok: 0.5363 - recall/ng: 0.6368
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.02775773464448429/final 
 {'loss': 1.2271252870559692, 'acc': 0.6339138746261597, 'precision/ok': 0.04250559210777283, 'precision/ng': 0.9785807132720947, 'recall/ok': 0.5362903475761414, 

17/17 [==============================] - 7s 428ms/step - loss: 1.0456 - acc: 0.6247 - precision/ok: 0.0403 - precision/ng: 0.9775 - recall/ok: 0.5202 - recall/ng: 0.6279
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-5_op-RMSprop_adv-True_ss-0.04511465105226098/final 
 {'loss': 1.045570969581604, 'acc': 0.6247352361679077, 'precision/ok': 0.0403251014649868, 'precision/ng': 0.9775429368019104, 'recall/ok': 0.5201612710952759, 'recall/ng': 0.6278787851333618}
17/17 [==============================] - 7s 415ms/step - loss: 0.1313 - acc: 0.9767 - precision/ok: 0.6838 - precision/ng: 0.9815 - recall/ok: 0.3750 - recall/ng: 0.9948
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-VGG16_lr-4_op-RMSprop_adv-True_ss-0.11923058234892084/final 
 {'loss': 0.13131672143936157, 'acc': 0.9767004251480103, 'precision/ok': 0.6838235259056091, 'precision/ng': 0.9814637899398804, 'recall/ok': 0.375, 'recall/ng

17/17 [==============================] - 8s 451ms/step - loss: 0.8583 - acc: 0.0292 - precision/ok: 0.0292 - precision/ng: 0.0000e+00 - recall/ok: 1.0000 - recall/ng: 0.0000e+00
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-DenseNet121_lr-5_op-RMSprop_adv-True_ss-0.058671337138140746/final 
 {'loss': 0.858282744884491, 'acc': 0.029183337464928627, 'precision/ok': 0.029183337464928627, 'precision/ng': 0.0, 'recall/ok': 1.0, 'recall/ng': 0.0}
17/17 [==============================] - 7s 392ms/step - loss: 0.8937 - acc: 0.0700 - precision/ok: 0.0303 - precision/ng: 0.9971 - recall/ok: 0.9960 - recall/ng: 0.0422
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/dc0503_relabeled_trans_r50/b-DenseNet121_lr-4_op-RMSprop_adv-True_ss-0.053488788195075965/final 
 {'loss': 0.8936576247215271, 'acc': 0.07001647353172302, 'precision/ok': 0.03031046688556671, 'precision/ng': 0.9971346855163574, 'recall/ok': 0.9959677457809448, 'recall/ng': 0.

#### ( first_tuned )

#### ( graph_regularization_training )

In [3]:
import tensorflow as tf
ly = tf.keras.layers
def BaseModel():
    inputs = {
        "image": tf.keras.Input((32, 32, 3), name="image"),
        "degree": tf.keras.Input([1], name="degree"),
    }
    e = inputs["degree"]
    e = ly.Embedding(9, 32*32*3,
                     input_length=1)(e)
    e = ly.Reshape((32, 32, 3))(e)

    x = inputs["image"]
    x = ly.Add()([x, e])
    x = ly.Conv2D(64, 7, padding="same")(x)
    x = ly.ReLU()(x)
    for filters in [128, 256, 512]:
        x = ly.Conv2D(filters, 3, padding="same")(x)
        x = ly.BatchNormalization()(x)
        x = ly.ReLU()(x)
        x = ly.MaxPool2D()(x)
    x = ly.Conv2D(1024, 3, padding="same")(x)
    x = ly.BatchNormalization()(x)
    x = ly.ReLU()(x)
    x = ly.GlobalAveragePooling2D()(x)
    x = ly.Dense(2)(x)
    x = ly.Activation("softmax", dtype="float32")(x)
    return tf.keras.Model(inputs=inputs, outputs=x)
base_model = BaseModel()
metrics = [
        tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name="precision/ok", class_id=0),
        tf.keras.metrics.Precision(name="precision/ng", class_id=1),
        tf.keras.metrics.Recall(name="recall/ok", class_id=0),
        tf.keras.metrics.Recall(name="recall/ng", class_id=1),
    ]
base_model.compile(
        "adam", tf.keras.losses.CategoricalCrossentropy(), metrics)

In [4]:
gr_h5 = '/data/aoi-wzs-p1-dip-fa-nvidia/experiments/graph_regularization_training/results/graph-label-component-099-20200513/adv_graph_reg_model_nbr_3.h5'
# base_model = tf.keras.models.load_model(gr_h5)
gr_h5 = '/data/aoi-wzs-p1-dip-fa-nvidia/experiments/graph_regularization_training/results/graph-label-component-099-20200513/adv_graph_reg_model_nbr_3'
base_model.load_weights(gr_h5)
base_model.evaluate(test_ds)

NameError: name 'test_ds' is not defined

In [8]:
gr_h5 = '/data/aoi-wzs-p1-dip-fa-nvidia/experiments/graph_regularization_training/results/graph-label-component-099-20200513/adv_graph_reg_model_nbr_3'
base_model.load_weights(gr_h5)
base_model.evaluate(test_ds)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1208 run_step  **
        outputs = model

#### ( ori metadata )

In [8]:
orim_h5_paths = [ '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/ori_metadata_trans_r50/b-VGG16_lr-4_op-adagrad_adv-True_ss-0.03654886093937098/final', 
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/ori_metadata_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.028690168136067472/final', 
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/ori_metadata_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.15538295745111494/final', 
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/ori_metadata_convs_r50/cc-4_ff-32_ks-2_s-3_mp-False_lr-4_op-adam_adv-True/ep_093-vl_0.104213-va_0.996125.h5', 
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/ori_metadata_convs_r50/cc-4_ff-32_ks-2_s-3_mp-False_lr-4_op-adam_adv-True/ep_094-vl_0.097888-va_0.996250.h5', 
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/ori_metadata_convs_r50/cc-4_ff-32_ks-2_s-3_mp-False_lr-4_op-adam_adv-True/final',
          ]
for e in map(tf.keras.models.load_model, orim_h5_paths):
    e.evaluate(test_ds)

17/17 [==============================] - 9s 514ms/step - loss: 0.1957 - acc: 0.9529 - precision/ok: 0.3721 - precision/ng: 0.9966 - recall/ok: 0.8911 - recall/ng: 0.9548


#### ( re_labeled )

In [ ]:
rl_h5_paths = [ '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.17064132560395412/final', 
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_trans_r50/b-VGG16_lr-4_op-adam_adv-True_ss-0.17064132560395412/ep_028-vl_0.001341-va_1.000000.h5', 
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_trans_r50/b-VGG16_lr-5_op-RMSprop_adv-True_ss-0.06535055175652027/final',
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_trans_r50/b-VGG16_lr-5_op-RMSprop_adv-True_ss-0.06535055175652027/ep_093-vl_0.003489-va_0.999375.h5',
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_trans_r50/b-VGG16_lr-5_op-adam_adv-True_ss-0.19638963328377465/final',
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_trans_r50/b-VGG16_lr-5_op-adam_adv-True_ss-0.19638963328377465/ep_090-vl_0.009936-va_1.000000.h5',
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_trans_r50/b-VGG16_lr-5_op-RMSprop_adv-True_ss-0.1472465226446634/final',
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_trans_r50/b-VGG16_lr-5_op-RMSprop_adv-True_ss-0.1472465226446634/ep_098-vl_0.013516-va_1.000000.h5',
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_convs_r50/cc-4_ff-8_ks-3_s-3_mp-False_lr-3_op-adam_adv-True/final',
            '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/re_labeled_convs_r50/cc-4_ff-8_ks-3_s-3_mp-False_lr-3_op-adam_adv-True/ep_099-vl_0.005794-va_0.999875.h5'
          ]
for e in map(tf.keras.models.load_model, rl_h5_paths):
    e.evaluate(test_ds)

17/17 [==============================] - 6s 382ms/step - loss: 0.3027 - acc: 0.9135 - precision/ok: 0.2012 - precision/ng: 0.9891 - recall/ok: 0.6613 - recall/ng: 0.9211
